In [1]:
# from pyDWSIMconnect import PyDWSIMconnect
from simulation import Simulation
import numpy as np
import time
from fobj import *
from scipy import optimize
from pprint import pprint

In [2]:
# Loading DWSIM simulation into Python (Simulation object)
sim_smr = Simulation(path2sim="C:\\Users\\lfsfr\\Desktop\\pyDWSIMopt\\sim\\SMR_2exp_phaseSep.dwxmz")
sim_smr.Add_refs()
sim_smr.Connect()

added refs
Simulation was loaded successfully


In [4]:
# Initial simulation setup
x0 = np.array( [0.25/3600, 0.70/3600, 1.0/3600, 1.10/3600, 1.80/3600, 2.50e5, 50.00e5, -40+273.15] )
sim_smr.fobj = lambda x: fobj8n_2exp(sim_smr, x)
sumW, mita = sim_smr.fobj(0.7*x0)
print(sim_smr.x,
sim_smr.f,
sim_smr.g)
sim_smr.interface.SaveFlowsheet(sim_smr.flowsheet,
        'C:\\Users\\lfsfr\\Desktop\\pyDWSIMopt\\sim\\SMR_2exp_phaseSep2.dwxmz',True)

[4.86111111e-05 1.36111111e-04 1.94444444e-04 2.13888889e-04
 3.50000000e-04 1.75000000e+05 3.50000000e+06 1.63205000e+02] 0.29672257055480267 -15.265529872662796


In [12]:
x_top = np.array( [0.252779/3600, 0.441063/3600, 1.411068/3600, 0.805952/3600, 1.851257/3600, 3.342832e5, 34.97221e5, -27.2958+273.15] )
args=(sim_smr)
xtol=0.01
ftol=0.01
maxiter=50 # +- 5 seconds per iteration
print("starting optimization")
iteration = 0
regularizer = np.array([1e4,1e4,1e4,1e4,1e4, 1e-5, 1e-6, 1e-2])
#rever \/
bounds_raw = np.array( [0.6*np.asarray(x0), 1.4*np.asarray(x0)] )
bounds_reg = regularizer*bounds_raw
bounds_reg[0][-1] = 153
bounds_reg[1][-1] = 233
print(bounds_reg)
bounds = optimize.Bounds(bounds_reg[0], bounds_reg[1])
g = lambda x: fobj8n_2exp(sim_smr,np.asarray(x)/regularizer)[1]
nonlinear_constraint = optimize.NonlinearConstraint(g, 3, np.inf, jac='2-point', hess=optimize.BFGS())
f = lambda x: fobj8n_2exp(sim_smr,np.asarray(x)/regularizer)[0]
# print(fobj8n_2exp(sim_smr,x_top))

starting optimization
[[  0.41666667   1.16666667   1.66666667   1.83333333   3.
    1.5          3.         153.        ]
 [  0.97222222   2.72222222   3.88888889   4.27777778   7.
    3.5          7.         233.        ]]


In [13]:
fobj8n_2exp(sim_smr, x0)
pprint(vars(sim_smr))

skipped with
{'f': 0.3933011539994796,
 'flowsheet': <DWSIM.UI.Desktop.Shared.Flowsheet object at 0x0000029F6F97F760>,
 'fobj': <function <lambda> at 0x0000029F73D52040>,
 'g': 6.617040555793466,
 'interface': <DWSIM.Automation.Automation2 object at 0x0000029F6F979C40>,
 'path': 'C:\\Users\\lfsfr\\Desktop\\pyDWSIMopt\\sim\\SMR_2exp_phaseSep.dwxmz',
 'path2dwsim': 'C:\\Users\\lfsfr\\AppData\\Local\\DWSIM7\\',
 'x': array([6.94444444e-05, 1.94444444e-04, 2.77777778e-04, 3.05555556e-04,
       5.00000000e-04, 2.50000000e+05, 5.00000000e+06, 2.33150000e+02])}


In [14]:
# Local optimization with trust-region -> working to some extent
result = optimize.minimize( f, np.asarray(x_top)*regularizer, 
            method='trust-constr', jac='2-point', hess=optimize.BFGS(),
            constraints=[nonlinear_constraint], bounds=bounds, callback=None,
            options={'verbose': 3, 
                     'xtol': xtol, 
                     'maxiter': maxiter, 
                     'finite_diff_rel_step': None, 
                     'initial_tr_radius': 0.1} )

skipped with
skipped with
skipped with
skipped with
skipped with
skipped with
skipped with
skipped with
skipped with
skipped with
| niter |f evals|CG iter|  obj func   |tr radius |   opt    |  c viol  | penalty  |barrier param|CG stop|
|-------|-------|-------|-------------|----------|----------|----------|----------|-------------|-------|
|   1   |   9   |   0   | +2.8781e-01 | 1.00e-01 | 3.37e-02 | 1.51e+02 | 1.00e+00 |  1.00e-01   |   0   |
skipped with
skipped with
skipped with
skipped with
skipped with
skipped with
skipped with
skipped with
skipped with
skipped with
|   2   |  18   |   1   | +2.8874e-01 | 7.00e-01 | 3.34e-02 | 1.51e+02 | 1.00e+00 |  1.00e-01   |   2   |
skipped with
skipped with
skipped with
skipped with
skipped with
skipped with
skipped with
skipped with
skipped with
skipped with
|   3   |  27   |   2   | +2.9449e-01 | 4.90e+00 | 3.09e-02 | 1.51e+02 | 1.00e+00 |  1.00e-01   |   2   |
skipped with
skipped with
skipped with
skipped with
skipped with
skipped with
sk

In [ ]:
print(f(result.x))
print(g(result.x))
# sim_smr.interface.SaveFlowsheet(sim_smr.flowsheet,sim_smr.path,True)
print(result.x)
# x_top
bounds


In [ ]:
# Global optimization with Differential Evolution
result = optimize.differential_evolution(f, bounds, constraints=[nonlinear_constraint], maxiter=20, seed=1234, disp=True, polish=False)